We are all very familiar with Logistic Regression and its optimization solution to minimize its cost function. But in reality, we may need some constraints. The following are two special constraints, please give: a) optimization solution; b) code of the solution process; c) sample code (you can use the sklearn data set or generate your own code)
1) Non-negative constraint: All coefficients of LR are required to be non-negative;
2) Order preserving constraint: The coefficient of LR is required to satisfy a1 >= a2 >= a3 ……


In the following, I edit a from-scratch Logistic Regression algorithm in the function coefficients_sgd to (1) ensure positive coefficients using absolute functions and (2) ensure positive and weights descending in strength from the first X term using absolute and np.sort functions 

For (1) and (2), I also vary the learning rate for each iteration to try to 'jiggle' gradient descent out of getting stuck in local maximas.

I provide my answers to the questions at the bottom. 

Pima Indians Diabetes Dataset
The Pima Indians dataset involves predicting the onset of diabetes within 5 years in Pima Indians given basic medical details.

Dataset File.
Dataset Details.
It is a binary classification problem, where the prediction is either 0 (no diabetes) or 1 (diabetes).

It contains 768 rows and 9 columns. All of the values in the file are numeric, specifically floating point values. Below is a small sample of the first few rows of the problem.

Predicting the majority class, the baseline performance on this problem is 65.098% classification accuracy.

We see later that our constriained algorithm produces accuracies of about 34%, about 48% drop in performance

# Positive Coefficients

In [39]:
# Logistic Regression on Diabetes Dataset
from random import seed
from random import randrange
from csv import reader
from math import exp
 
# Load a CSV file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset
 
# Convert string column to float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())
 
# Find the min and max values for each column
def dataset_minmax(dataset):
    minmax = list()
    for i in range(len(dataset[0])):
        col_values = [row[i] for row in dataset]
        value_min = min(col_values)
        value_max = max(col_values)
        minmax.append([value_min, value_max])
    return minmax
 
# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
    for row in dataset:
        for i in range(len(row)):
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])
 
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split
 
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0
 
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores
 
# Make a prediction with coefficients
def predict(row, coefficients):
    yhat = coefficients[0]
    for i in range(len(row)-1):
        yhat += coefficients[i + 1] * row[i]
    return 1.0 / (1.0 + exp(-yhat))
 
# Estimate logistic regression coefficients using stochastic gradient descent
def coefficients_sgd(train, l_rate, n_epoch):
    coef = [0.0 for i in range(len(train[0]))]
    for epoch in range(n_epoch):
        for row in train:
            yhat = predict(row, coef)
            error = row[-1] - yhat
            # I use absolute functions here to ensure positive coefficients
            coef[0] = abs(coef[0]) + l_rate * error * yhat * (1.0 - yhat)
            for i in range(len(row)-1):
                coef[i + 1] = coef[i + 1] + l_rate * error * yhat * (1.0 - yhat) * row[i]
                coef = [abs(i) for i in coef]
                
    return coef
 
# Linear Regression Algorithm With Stochastic Gradient Descent
def logistic_regression(train, test, l_rate, n_epoch):
    predictions = list()
    coef = coefficients_sgd(train, l_rate, n_epoch)
    print("coefficients for each kfold step are positive - " + str(coef))
    for row in test:
        yhat = predict(row, coef)
        yhat = round(yhat)
        predictions.append(yhat)
        
    return(predictions)
 
def run_test1():
    # Test the logistic regression algorithm on the diabetes dataset
    seed(1)
    # load and prepare data
    filename = 'pima-indians-diabetes.csv'
    dataset = load_csv(filename)
    for i in range(len(dataset[0])):
        str_column_to_float(dataset, i)
    # normalize
    minmax = dataset_minmax(dataset)
    normalize_dataset(dataset, minmax)
    # evaluate algorithm
    n_folds = 5
    l_rate = 0.1
    n_epoch = 100
    scores = evaluate_algorithm(dataset, logistic_regression, n_folds, l_rate, n_epoch)
    print('Scores: %s' % scores)
    print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

In [40]:
run_test1()

coefficients for each kfold step are positive - [0.0003551784706798695, 0.02881803671570453, 0.0006707048505025731, 0.002852099842011303, 0.005523343823740465, 0.003071089742031467, 0.006896414169791931, 0.008424637707320935, 0.01470274052357582]
coefficients for each kfold step are positive - [0.00042026942840225033, 0.028677363048408233, 0.0007226294738801355, 0.0027682902867737077, 0.00553033782415361, 0.0030750891619692323, 0.006934607966704234, 0.008418516521555114, 0.014683627261389784]
coefficients for each kfold step are positive - [0.0004267357993742553, 0.02865753474554763, 0.0006945631820236709, 0.0028425954602073486, 0.005547371948074472, 0.0030456977933912346, 0.006876130000905141, 0.008423095398940485, 0.01470706144164734]
coefficients for each kfold step are positive - [0.00035976343949581206, 0.028693088178692587, 0.0006066505086812632, 0.002755240624559864, 0.0055227793057408235, 0.0029247510521121156, 0.006863015458226159, 0.00839366754465008, 0.014593167770915856]
co

# Positive and Descending Coefficients

In [43]:
# Logistic Regression on Diabetes Dataset
from random import seed
from random import randrange
from csv import reader
from math import exp
 
# Load a CSV file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset
 
# Convert string column to float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())
 
# Find the min and max values for each column
def dataset_minmax(dataset):
    minmax = list()
    for i in range(len(dataset[0])):
        col_values = [row[i] for row in dataset]
        value_min = min(col_values)
        value_max = max(col_values)
        minmax.append([value_min, value_max])
    return minmax
 
# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
    for row in dataset:
        for i in range(len(row)):
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])
 
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split
 
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0
 
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores
 
# Make a prediction with coefficients
def predict(row, coefficients):
    yhat = coefficients[0]
    for i in range(len(row)-1):
        yhat += coefficients[i + 1] * row[i]
    return 1.0 / (1.0 + exp(-yhat))
 
# Estimate logistic regression coefficients using stochastic gradient descent
def coefficients_sgd(train, l_rate, n_epoch):
    coef = [0.0 for i in range(len(train[0]))]
    for epoch in range(n_epoch):
        for row in train:
            yhat = predict(row, coef)
            error = row[-1] - yhat
            
            # I use absolute functions here and a sort function to ensure that coefficients are positive and descending in order
            coef[0] = abs(coef[0]) + l_rate * error * yhat * (1.0 - yhat)
            for i in range(len(row)-1):
                coef[i + 1] = coef[i + 1] + l_rate * error * yhat * (1.0 - yhat) * row[i]
                coef = [abs(i) for i in coef]
                coef = sorted(coef, reverse = True)
                
    return coef
 
# Linear Regression Algorithm With Stochastic Gradient Descent
def logistic_regression(train, test, l_rate, n_epoch):
    predictions = list()
    coef = coefficients_sgd(train, l_rate, n_epoch)
    print("coefficients for each kfold step are positive AND descending in order - " + str(coef))
    for row in test:
        yhat = predict(row, coef)
        yhat = round(yhat)
        predictions.append(yhat)
        
    return(predictions)
 
def run_test2():
    # Test the logistic regression algorithm on the diabetes dataset
    seed(1)
    # load and prepare data
    filename = 'pima-indians-diabetes.csv'
    dataset = load_csv(filename)
    for i in range(len(dataset[0])):
        str_column_to_float(dataset, i)
    # normalize
    minmax = dataset_minmax(dataset)
    normalize_dataset(dataset, minmax)
    # evaluate algorithm
    n_folds = 5
    l_rate = 0.5
    n_epoch = 100
    scores = evaluate_algorithm(dataset, logistic_regression, n_folds, l_rate, n_epoch)
    print('Scores: %s' % scores)
    print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

In [44]:
run_test2()

coefficients for each kfold step are positive AND descending in order - [0.06249187279469407, 0.048169059868665154, 0.04084932612441459, 0.029452733550942738, 0.025555252374678063, 0.020964617954868814, 0.02009444310507721, 0.013866394913012697, 0.004580970878454833]
coefficients for each kfold step are positive AND descending in order - [0.06249102076815966, 0.04817517938682098, 0.04084422447548834, 0.029457379861677298, 0.025556363054482032, 0.02096500091825213, 0.020091908689768297, 0.013868753572509055, 0.004580786449692217]
coefficients for each kfold step are positive AND descending in order - [0.06249993902786857, 0.048186151179277395, 0.0408462006349529, 0.029459307431665, 0.025568237012807898, 0.02097462870068146, 0.020088100499477922, 0.013858635355591113, 0.004594322232850927]
coefficients for each kfold step are positive AND descending in order - [0.06251911625761436, 0.04817865823570523, 0.040872239214982564, 0.029455661876163275, 0.025512772125654518, 0.020984939769950528

One more question: If there is no constraint, is LR a global optimization algorithm or a local one? why? When constraints 1) or 2) are imposed, global or local? why?

QN: IS LR A GLOBAL OPTIMIZATION ALGORITHM

Global optimization refers to finding the optimal value of a given function among all possible solutions whereas local optimization finds the optimal solution for a specific region of the search space (or global optima for problems with no local optima)

The two can intersect, when the local-minimum is also the global minimum. This is known as convexity. 

Logistic Regression is a local optimization algorithm intended to locate a local optima. It operates on a single candidate solution and involves iteratively making small changes to the candidate solution and evaluating the change to see if it leads to an improvement and is taken as the new candidate solution. 

However, despite it being a local optimization algorithm, it can locate the global optimum (1) if the local optima is the global optima or (2) the region being searched contains the global optima. 

In contrast, a global optimization algorithm is intended to locate the global optima. It will traverse the ENTIRE input space to find the optima. It does so by managing population of candidate solutions from which new candidate solutions are iteratively generated. As we see from the above solution, Logistic Regression does not fit this description as only one candidate solution is iterated and evaluated. 

QN: WHEN CONSTRAINTS ARE IMPOSED, GLOBAL OR LOCAL? 

When contraints are imposed, LR REMAINS a local optimization algorithm. In FACT, it becomes even MORE local, because we have (in the case of our constraints 1 and 2), drastically reduced the input search space that we can traverse. What has happened is that we have reduced the convexity of the problem, where we are no longer guaranteed that the local minimum is also the global minimum.

So LR was never a global optimization algorithm, just a local one that is often able to find the global minimum by consequence. With constraints, we have possibly broken this link, and may have to be satisfied with local minimas that are non-global optimal.